# DS_Automation_Assignment_Oussama_Ennaciri

Using our prepared churn data from week 2:
- use pycaret to find an ML algorithm that performs best on the data
    - Choose a metric you think is best to use for finding the best model; by default, it is accuracy but it could be AUC, precision, recall, etc. The week 3 FTE has some information on these different metrics.
- save the model to disk
- create a Python script/file/module with a function that takes a pandas dataframe as an input and returns the probability of churn for each row in the dataframe
    - your Python file/function should print out the predictions for new data (new_churn_data.csv)
    - the true values for the new data are [1, 0, 0, 1, 0] if you're interested
- test your Python module and function with the new data, new_churn_data.csv
- write a short summary of the process and results at the end of this notebook
- upload this Jupyter Notebook and Python file to a Github repository, and turn in a link to the repository in the week 5 assignment dropbox

## Preparing the Environment and Loading the Data

We started by preparing the virtual environment msds. We created the virtual environment via terminal using the following commands.

In [68]:
#!conda create -n msds python=3.10.14 -y
#!conda activate msds

Then, we loaded the prepared data.

In [16]:
import pandas as pd

df = pd.read_csv('prepped_churn_data.csv', index_col='customerID')
df

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,charge_per_tenure
customerID,,,,,,,,
7590-VHVEG,1,0,0,3,29.85,29.85,0,29.850000
5575-GNVDE,34,1,1,2,56.95,1889.50,0,55.573529
3668-QPYBK,2,1,0,2,53.85,108.15,1,54.075000
7795-CFOCW,45,0,1,1,42.30,1840.75,0,40.905556
9237-HQITU,2,1,0,3,70.70,151.65,1,75.825000
...,...,...,...,...,...,...,...,...
6840-RESVB,24,1,1,2,84.80,1990.50,0,82.937500
2234-XADUH,72,1,1,0,103.20,7362.90,0,102.262500
4801-JZAZL,11,0,0,3,29.60,346.45,0,31.495455


## AutoML with pycaret

In order to use pycart for autoML, we need to install it and then import the required packages.

In [3]:
conda install -c conda-forge pycaret -y

Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
from pycaret.classification import setup, compare_models, predict_model, save_model, load_model

We start by preparing the autoML and defining the target "Churn".

In [3]:
automl = setup(df, target='Churn')

,Description,Value
0,Session id,6679
1,Target,Churn
2,Target type,Binary
3,Original data shape,"(7032, 8)"
4,Transformed data shape,"(7032, 8)"
5,Transformed train set shape,"(4922, 8)"
6,Transformed test set shape,"(2110, 8)"
7,Numeric features,7
8,Preprocess,True
9,Imputation type,simple


Next, we run the autoML to compare the ML models and find the best one.

In [18]:
best_model = compare_models(sort='AUC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7895,0.8374,0.4969,0.6354,0.5551,0.4205,0.4274,0.1200
lr,Logistic Regression,0.7960,0.8351,0.5221,0.6451,0.5752,0.4433,0.4488,0.3550
ada,Ada Boost Classifier,0.7926,0.8346,0.5099,0.6408,0.5659,0.4322,0.4383,0.0370
lightgbm,Light Gradient Boosting Machine,0.7881,0.8267,0.5000,0.6291,0.5549,0.4188,0.4247,0.0220
ridge,Ridge Classifier,0.7913,0.8232,0.4541,0.6579,0.5345,0.4064,0.4194,0.0060
lda,Linear Discriminant Analysis,0.7909,0.8232,0.5084,0.6346,0.5628,0.4280,0.4335,0.0100
qda,Quadratic Discriminant Analysis,0.7501,0.8156,0.7247,0.5217,0.6063,0.4304,0.4433,0.0080
nb,Naive Bayes,0.7430,0.8139,0.7140,0.5119,0.5958,0.4149,0.4278,0.0090
rf,Random Forest Classifier,0.7781,0.8074,0.4778,0.6080,0.5335,0.3909,0.3967,0.0880
et,Extra Trees Classifier,0.7653,0.7884,0.4808,0.5715,0.5215,0.3677,0.3705,0.0700


We notice that the logistic regression model has the highest accuracy, MCC, and kappa measures, and the gradient boosting classifier has the highest AUC measure.

In [19]:
best_model

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=6679, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

Using our AutoML and sorting by AUC we deduct that the best model to use is the Gradient Boosting Classifier model.

In [20]:
df.iloc[-2:-1].shape

(1, 8)

In [21]:
predict_model(best_model, df.iloc[-2:-1])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,1.0000,0,1.0000,1.0000,1.0000,nan,0.0000


,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,charge_per_tenure,Churn,prediction_label,prediction_score
customerID,,,,,,,,,,
8361-LTMKD,4,1,0,2,74.400002,306.600006,76.650002,1,1,0.5764


We use the model to make predictions on our data. 
We are selecting the second to last row, but using the indexing `[-2:-1]`
The model predicted churn as indicated by the prediction label and the score of 57.64% which is good because in that specific case the customer did churn.

## Saving and Loading our Model

Next, we will save our trained model so we can use it in a Python file later.

In [22]:
save_model(best_model, 'GBC')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['tenure', 'PhoneService',
                                              'Contract', 'PaymentMethod',
                                              'MonthlyCharges', 'TotalCharges',
                                              'charge_per_tenure'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean'))),
                 ('c...
                                             criterion='f

In [23]:
import pickle

with open('GBC_model.pk', 'wb') as f:
    pickle.dump(best_model, f)

Here, we open a file with the name `GBC_model.pk` for writing, and we save that file object in the variable `f`. After that, we load it.

In [24]:
with open('GBC_model.pk', 'rb') as f:
    loaded_model = pickle.load(f)

In [25]:
new_data = df.iloc[-2:-1].copy()
new_data.drop('Churn', axis=1, inplace=True)
loaded_model.predict(new_data)

array([1], dtype=int8)

In [27]:
loaded_GBC = load_model('GBC')

Transformation Pipeline and Model Successfully Loaded


In [28]:
predict_model(loaded_GBC, new_data)

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,charge_per_tenure,prediction_label,prediction_score
customerID,,,,,,,,,
8361-LTMKD,4,1,0,2,74.400002,306.600006,76.650002,1,0.5764


## Making a Python module to make predictions

In [29]:
from IPython.display import Code

Code('predict_churn.py')

import pandas as pd
from pycaret.classification import predict_model, load_model

def load_data(filepath):
    """
    Loads churn data into a DataFrame from a string filepath.
    """
    df = pd.read_csv(filepath, index_col='customerID')
    return df


def make_predictions(df):
    """
    Uses the pycaret best model to make predictions on data in the df dataframe.
    """
    model = load_model('GBC')
    predictions = predict_model(model, data=df)

    # Check the column names
    print(predictions.columns)
    
    # Rename 'prediction_label' to 'churn_prediction' if it exists
    if 'prediction_label' in predictions.columns:
        predictions.rename(columns={'prediction_label': 'churn_prediction'}, inplace=True)
        
        # Replace values in the new column
        predictions['churn_prediction'].replace({1: 'Churn', 0: 'No Churn'}, inplace=True)
        
        return predictions['churn_prediction']
    else:
        raise KeyError("The 'prediction_label' column was not found in the predictions DataFrame")


if __name__ == "__main__":
    df = load_data('new_churn_data.csv')
    predictions = make_predictions(df)
    print('predictions:')
    print(predictions)

We finally run the python script which will predict churn based on the chosen pre-trained model.

In [30]:
%run predict_churn.py

Transformation Pipeline and Model Successfully Loaded


Index(['tenure', 'PhoneService', 'Contract', 'PaymentMethod', 'MonthlyCharges',
       'TotalCharges', 'charge_per_tenure', 'prediction_label',
       'prediction_score'],
      dtype='object')
predictions:
customerID
9305-CKSKC    No Churn
1452-KNGVK    No Churn
6723-OKKJM    No Churn
7832-POPKP    No Churn
6348-TACGU    No Churn
Name: churn_prediction, dtype: object


# Summary

We started by preparing the virtual environment ‘msds’. We created the virtual environment via terminal and activated it. Then, we loaded the prepared data.

In order to use pycart for autoML, we installed it via Honda and then imported the required packages ‘setup, compare_models, predict_model, save_model, load_model’.

We prepared the autoML and defined the target "Churn".
Next, we ran the autoML to compare the ML models and find the best one. We notice that the logistic regression model has the highest accuracy, MCC, and kappa measures, but gradient boosting classifier has the highest AUC measure.

Using our AutoML, and sorting by AUC, we deducted that the best model to use is the Gradient Boosting Classifier model.

We used the model to make predictions on our data. We are selecting a row ‘the second to last row’, but using the indexing `[-2:-1]`. The model predicted churn as indicated by the prediction label and the score of 57.64%. This is good because in that specific case the customer did churn. We will save our trained model so we can use it in a Python file later.

We finally ran the python script which will predict churn based on the chosen pre-trained model. The model predicted the following results:

9305-CKSKC ;   No Churn
1452-KNGVK ;   No Churn
6723-OKKJM ;   No Churn
7832-POPKP ;   No Churn
6348-TACGU ;   No Churn

The Gradient Boosting Classifier model predicted none of the clients churned, which is not accurate as the true data shows that two clients churned. This is an accuracy percentage of 60%.